In [3]:
import numpy as np
from cellpose import models, core, io
from spotiflow.model import Spotiflow
from pathlib import Path
from pathlib import Path
import os
import apoc
from tqdm import tqdm
import pyclesperanto_prototype as cle 
from tifffile import imwrite, imread
import pandas as pd
from utils import list_images, read_image

io.logger_setup() # run this to get printing of progress

#Check if notebook has GPU access
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

#Activate .cle GPU acceleration
cle.select_device("gpu")

#Load pre-trained Cellpose-SAM and Spotiflow models
model = models.CellposeModel(gpu=True)
spotiflow_model = Spotiflow.from_pretrained("general")

creating new log file
2025-09-01 15:30:02,571 [INFO] WRITING LOG OUTPUT TO C:\Users\adiez_cmic\.cellpose\run.log
2025-09-01 15:30:02,571 [INFO] 
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.10.18 
torch version:  	2.5.0
2025-09-01 15:30:02,572 [INFO] ** TORCH CUDA version installed and working. **
2025-09-01 15:30:02,573 [INFO] ** TORCH CUDA version installed and working. **
2025-09-01 15:30:02,574 [INFO] >>>> using GPU (CUDA)
2025-09-01 15:30:03,708 [INFO] >>>> loading model C:\Users\adiez_cmic\.cellpose\models\cpsam
INFO:spotiflow.model.spotiflow:Loading pretrained model: general
2025-09-01 15:30:04,060 [INFO] Loading pretrained model: general


In [ ]:
cl_filename = "./raw_data/SD_DAPI_Mtb_detection_training/Mtb_segmenter.cl"
mtb_segmenter = apoc.ObjectSegmenter(opencl_filename=cl_filename)

# Create a list of subdirectories containing the Nuc string (stained for nuclei)
main_directory_path = Path("X:\Lisa\siMtb screen I_LØ")

folders_with_nuc = [
    name for name in os.listdir(main_directory_path)
    if os.path.isdir(os.path.join(main_directory_path, name)) and "Nuc" in name
]

print(folders_with_nuc)

In [ ]:
for folder in tqdm(folders_with_nuc):

    print(f"Analyzing Plate: {folder}")

    # Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
    directory_path = Path(f"X:\Lisa\siMtb screen I_LØ\{folder}")

    # Iterate through the .czi and .nd2 files in the directory
    images = list_images(directory_path)

    # Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
    slicing_factor_xy = None # Use 2 or 4 for downsampling in xy (None for lossless)

    #TODO: Substract uneven and remove background from BF by obtaining the median of all BF channels 

    try:

        bf_correction = imread(directory_path / "bf_correction.tiff")

    except FileNotFoundError:

        # Create an empty list to store the brightfield images from each well
        bf_arrays = []

        # Read all images, extract the brightfield channel and calculate the mean to correct illumination and remove dust spots
        for image in tqdm(images):

            # Read image, apply slicing if needed and return filename and img as a np array
            img, filename = read_image(image, slicing_factor_xy, log=False)

            # Extract brighfield slice
            bf_channel = img[4]

            # Add it to the bf_arrays iterable
            bf_arrays.append(bf_channel)

        # Create a stack containing all bf images
        bf_stack = np.stack(bf_arrays, axis=0)

        # Calculate the median to retain the common structures (spots, illumination)
        bf_correction = np.median(bf_stack, axis=0)
        del bf_stack

        # Store brightfield correction as .tiff to avoid recalculating it everytime
        imwrite(directory_path / "bf_correction.tiff",bf_correction)

    stats = []

    for image in tqdm(images):

        # Read image, apply slicing if needed and return filename and img as a np array
        img, filename = read_image(image, slicing_factor_xy)

        # Extract plate number and well_id
        plate_nr = filename.split("_")[0]
        well_id = filename.split("-")[1][:2]

        cytoplasm_labels, flows, styles = model.eval(np.stack((img[[0,1]].sum(axis=0), (img[4] - bf_correction)), axis=0), niter=1000) # need to check the arguments

        mtb_labels = mtb_segmenter.predict(img[3])
        mtb_labels = cle.pull(mtb_labels)

        # Convert mtb_labels to boolean mask
        mtb_boolean = mtb_labels.astype(bool)

        # Use NumPy's indexing to identify labels that intersect with mtb_boolean (bacterial mask)
        infected_labels = np.unique(cytoplasm_labels[mtb_boolean])
        infected_labels = infected_labels[infected_labels != 0]

        infected_mask = np.isin(cytoplasm_labels, infected_labels)
        non_infected_mask = np.isin(cytoplasm_labels, infected_labels, invert=True)
        infected_cytoplasm = np.where(infected_mask, cytoplasm_labels, 0).astype(cytoplasm_labels.dtype)
        non_infected_cytoplasm = np.where(non_infected_mask, cytoplasm_labels, 0).astype(cytoplasm_labels.dtype)

        infected_cells = len(np.unique(infected_cytoplasm)) - (0 in infected_cytoplasm)
        non_infected_cells = len(np.unique(non_infected_cytoplasm)) - (0 in non_infected_cytoplasm)
        total_cells = cytoplasm_labels.max()

        # Calculate percentage of infected cells
        perc_inf_cells = round(infected_cells / total_cells * 100, 2) if total_cells > 0 else 0

        print(f"Non-infected: {non_infected_cells}")
        print(f"Infected: {infected_cells}")
        print(f"Total cells: {total_cells}")
        print(f"Percentage infected:{perc_inf_cells}")

        # Create a dictionary containing all extracted info per image
        stats_dict = {
                    "plate": plate_nr,
                    "well_id": well_id,
                    "total_nr_cells": total_cells,
                    "infected": infected_cells,
                    "non-infected": non_infected_cells,
                    "%_inf_cells": perc_inf_cells 
        }

        # Append the current data point to the stats_list
        stats.append(stats_dict)

    # Transform into a dataframe to store it as .csv later
    df = pd.DataFrame(stats)

    df.to_csv(f"./results_infection_{plate_nr}.csv")